In [1]:
import lsst.sims.maf
lsst.sims.maf.__version__

'2.4.2.sims'

In [2]:
%matplotlib inline   
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

#### Set Parameters

In [4]:
filters = ['u','g','r','i','z','y']
colors={'u': 'cyan','g': 'g','r': 'y',
        'i': 'r','z': 'm', 'y': 'k'}

In [5]:
# Set RA, Dec for a single point in the sky.
# The input ra/dec values should be in DEGREES (even for v3 databases). 
ra = [180.0, 10]
dec = [-50.0, 10]
# SNR limit (Don't use points below this limit)
snrLimit = 5.
# Demand this many points above SNR limit before plotting LC
nPtsLimit = 6

## Run engima_1189

In [6]:
# setup database connection
outDir ='LightCurve'
dbFile = '/home/data/enigma_1189_sqlite.db'
opsimdb = db.OpsimDatabase(dbFile)
resultsDb = db.ResultsDb(outDir=outDir)

In [7]:
from numpy import amin as m

In [8]:
# retrive min inter_night gap, and observation history with the input of database file name and
# arrays of RA and DEC
def find_min_gap(dbFile, ra, dec):
    # While we're in transition between opsim v3 and v4, this may be helpful:
    print("{dbFile} is an opsim version {version} database".format(dbFile=dbFile, version=opsimdb.opsimVersion))
    if opsimdb.opsimVersion == "V3":
        # For v3 databases:
        mjdcol = 'expMJD'
        degrees = False
    else:
        # For v4 and alternate scheduler databases.
        mjdcol = 'observationStartMJD'
        degrees = True
    # Note that astro-lsst-01_2022.db is a v4 database.
    
    metric = metrics.cadenceMetrics.IntraNightGapsMetric(reduceFunc=np.amin, timeCol=mjdcol)
    metric_pass = metrics.simpleMetrics.PassMetric(cols=['filter','fiveSigmaDepth', mjdcol, 'expDate'])
    slicer = slicers.UserPointsSlicer(ra, dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg=degrees)
    sql = '' #'night < 365'
    bundle = metricBundles.MetricBundle(metric,slicer,sql)
    date_bundle = metricBundles.MetricBundle(metric_pass, slicer, sql)
    # In case you are using a dither stacker, we can check what columns are actually being pulled from the database.
    print(bundle.dbCols)
    
    # create metric bundle and return min gap
    bg = metricBundles.MetricBundleGroup({'gap': bundle, 'date':date_bundle}, opsimdb, outDir=outDir, resultsDb=resultsDb)
    bg.runAll()
    return bg

In [9]:
# assign returned value to b
b = find_min_gap(dbFile, ra, dec)

/home/data/enigma_1189_sqlite.db is an opsim version V3 database
{'fieldDec', 'night', 'expMJD', 'fieldRA'}
Querying database Summary with no constraint for columns ['night', 'fiveSigmaDepth', 'fieldRA', 'fieldDec', 'filter', 'expMJD', 'expDate'].
Found 2469307 visits
Running:  ['gap', 'date']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [10]:
# min intra_night gap in days
b.bundleDict['gap'].metricValues.data

array([ 0.01068,  0.02136])

In [11]:
# corresponding pointing history
b.bundleDict['date'].metricValues.data

array([ array([(  24,  21.349657,  3.141593, -0.845448, 'y',  49377.277517,   2097577),
       (  24,  21.34078 ,  3.141593, -0.894354, 'y',  49377.278422,   2097655),
       (  24,  21.300703,  3.141593, -0.894354, 'y',  49377.286559,   2098358),
       ...,
       (3648,  24.562715,  3.141593, -0.894354, 'r',  53001.29497 , 315212685),
       (3649,  24.617959,  3.141593, -0.894354, 'r',  53002.306357, 315300069),
       (3649,  24.676865,  3.141593, -0.894354, 'r',  53002.327018, 315301854)],
      dtype=(numpy.record, [('night', '<i8'), ('fiveSigmaDepth', '<f8'), ('fieldRA', '<f8'), ('fieldDec', '<f8'), ('filter', '<U256'), ('expMJD', '<f8'), ('expDate', '<i8')])),
       array([( 193,  24.616138,  0.196871,  0.187126, 'g',  49546.334277,  16704081),
       ( 193,  24.675719,  0.196871,  0.187126, 'g',  49546.356078,  16705965),
       ( 193,  24.772122,  0.196871,  0.187126, 'g',  49546.370937,  16707248),
       ( 193,  24.766633,  0.196871,  0.187126, 'g',  49546.39067 ,  167089